# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: PEFT token classification
* Model: distilbert-base-uncased
* Evaluation approach: "evaluate" and also try to use hugging face "Trainer"
* Fine-tuning dataset: Hugging Face library "datasets"

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install transformers datasets peft

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset

# Load the pre-trained model and tokenizer
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load a dataset for evaluation
dataset = load_dataset("ag_news", split="test[:1%]")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True, max_length=512)

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [4]:
import torch

def evaluate_model(model, dataset):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for example in dataset:
            inputs = {key: example[key].unsqueeze(0) for key in ['input_ids', 'attention_mask']}
            labels = example['label'].unsqueeze(0)
            
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=-1)
            
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

            # Break after processing 100 examples to make it even faster
            if total >= 100:
                break

    return correct / total

# Evaluate the original model
original_model_accuracy = evaluate_model(model, encoded_dataset)
print(f"Original Model Accuracy: {original_model_accuracy:.2f}")

Original Model Accuracy: 0.22


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [5]:
from peft import LoraConfig, TaskType

# Create a PEFT config
config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin", "out_lin", "ffn.lin1", "ffn.lin2"]
)

In [6]:
from peft import get_peft_model

# Convert the model into a PEFT model
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 1,777,160 || all params: 68,140,040 || trainable%: 2.6080994375700395


In [7]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Define a simple training loop
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset,
)

# Train the model
trainer.train()

# Save the trained PEFT model
lora_model.save_pretrained("distilbert-lora")

Step,Training Loss


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [9]:
from transformers import AutoModelForSequenceClassification
from peft import PeftModel, PeftConfig

# Load the base model
base_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

# Load the PEFT configuration
peft_config = PeftConfig.from_pretrained("distilbert-lora")

# Load the trained PEFT model
lora_model = PeftModel.from_pretrained(base_model, "distilbert-lora")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Evaluate the fine-tuned PEFT model
fine_tuned_model_accuracy = evaluate_model(lora_model, encoded_dataset)
print(f"Fine-Tuned Model Accuracy: {fine_tuned_model_accuracy:.2f}")

# Compare the performance
print(f"Original Model Accuracy: {original_model_accuracy:.2f}")
print(f"Fine-Tuned Model Accuracy: {fine_tuned_model_accuracy:.2f}")

Fine-Tuned Model Accuracy: 0.21
Original Model Accuracy: 0.22
Fine-Tuned Model Accuracy: 0.21
